In [ ]:

import numpy as np
from delta_hes_model import *





In [10]:
d = np.arange(10)
P = 10 
Q = 1

conn_matrix = connectivitymatrix(P, Q)


d_neighbours = np.sum(conn_matrix * d.flatten(), axis=1)
print(d)
print(d_neighbours)

[0 1 2 3 4 5 6 7 8 9]
[ 1.  2.  4.  6.  8. 10. 12. 14. 16.  8.]


In [4]:
import numpy as np
import matplotlib.pyplot as plt

t = np.arange(10000)*0.1
n = 20
x = np.zeros([len(t),n])
for i in range(n):
    x[:,i] = np.sin(t + np.pi/4*i) #phase shift each x by n
    
# Calculate the phase gradient 
from scipy.signal import hilbert 

analytic_signal = hilbert(x[15,:])
instantaneous_phase = np.angle(analytic_signal)  # phase at each time
print(instantaneous_phase)

phase_gradient = np.gradient(instantaneous_phase)
print('phase gradient', phase_gradient)

unwrapped_phase = np.unwrap(instantaneous_phase)

# Option 1: Compute gradient and sum it
dx = 1  # spatial step size
phase_gradient = np.gradient(unwrapped_phase, dx)
total_phase_shift = np.sum(phase_gradient)
print(total_phase_shift/np.pi)

[-0.51684466  0.83260341  1.49352514  2.22568772  3.10557029 -2.46789324
 -1.64005099 -0.84435759 -0.09001875  0.70430124  1.49851415  2.31744573
  3.09615914 -2.3626322  -1.63915801 -0.94922512 -0.12924277  0.42295865
  1.48945421 -2.62474799]
phase gradient [ 1.34944807  1.0051849   0.69654215  0.80602257 -2.34679048 -2.37281064
  0.81176782  0.77501612  0.77432942  0.79426645  0.80657224  0.79882249
 -2.34003896 -2.36765857  0.70670354  0.75495762  0.68609189  0.80934849
 -1.52385332 -4.1142022 ]
5.88900924514885


In [ ]:
# Take a snapshot in time (e.g., t = 0)
snapshot = x[10, :]  # signal across cells at t = 0

# Use Hilbert transform to get spatial phase at that time point
analytic_signal = hilbert(snapshot)
instantaneous_phase = np.angle(analytic_signal)
unwrapped_phase = np.unwrap(instantaneous_phase)

for i in range(n):
    if unwrapped_phase[i] < 0:
        unwrapped_phase *= -1

plt.figure()
plt.plot(snapshot)
plt.plot(analytic_signal)

# Compute spatial gradient
dx = 1
phase_gradient = np.gradient(unwrapped_phase, dx)
print(phase_gradient.shape)
plt.figure()
plt.plot(phase_gradient)
# Compute total phase shift across all cells
total_phase_shift = np.sum(phase_gradient)

print(f"Total Phase Shift: {total_phase_shift:.2f} rad")
print(f"Total Phase Shift in pi units: {total_phase_shift/np.pi:.2f} π")
print(f"Expected: {(n-1)*(np.pi/4):.2f} rad = {(n-1)/4:.2f} π")
print(f'Begin - end unwrapped phase: {(unwrapped_phase[-1]-unwrapped_phase[0])/np.pi:.2f} pi')
# Plot
plt.figure()
plt.plot(unwrapped_phase, marker='o')
plt.title("Unwrapped Phase Across Cells at t = 0")
plt.xlabel("Cell Index")
plt.ylabel("Phase (radians)")
plt.grid(True)
plt.show()

In [ ]:
from scipy.signal import find_peaks

# Store peak times for each cell
first_peak_times = np.zeros(n)

for i in range(n):
    signal = x[:, i]

    # Find peaks
    peaks, _ = find_peaks(signal, distance=20)  # adjust `distance` as needed

    if len(peaks) > 0:
        first_peak_times[i] = t[peaks[0]]
    else:
        first_peak_times[i] = np.nan  # or handle missing peaks

# Estimate period from one representative cell (or global average if needed)
ref_peaks, _ = find_peaks(x[:, 0], distance=20)
ref_periods = np.diff(t[ref_peaks])
mean_period = np.mean(ref_periods)

# Calculate time differences
time_diffs = first_peak_times - first_peak_times[0]

# Convert to phase: φ = Δt / T * 2π (accumulates!)
phase_shifts = time_diffs / mean_period * 2 * np.pi

# Now accumulate the phase shift (additive)
accumulated_phase_shift = np.cumsum(phase_shifts)  # cumulative sum across cells

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(accumulated_phase_shift, marker='o', label='Cumulative Phase Shift')
plt.axhline(2*np.pi, color='gray', linestyle='--', label='2π')
plt.axhline(4*np.pi, color='gray', linestyle='--', label='4π')
plt.xlabel("Cell Index")
plt.ylabel("Accumulated Phase Shift (radians)")
plt.title("Cumulative Phase Shift Across Cells")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Print the total accumulated phase shift
total_phase_shift = accumulated_phase_shift[-1]
print(f"Total Accumulated Phase Shift: {total_phase_shift:.2f} rad ≈ {total_phase_shift/np.pi:.2f} π")

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(phase_shifts, marker='o', label='Accumulated Phase Shift')
plt.axhline(2*np.pi, color='gray', linestyle='--', label='2π')
plt.axhline(4*np.pi, color='gray', linestyle='--', label='4π')
plt.xlabel("Cell Index")
plt.ylabel("Phase Shift (radians)")
plt.title("Accumulative Phase Shift Across Cells")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

print(f"Total Phase Shift: {phase_shifts[-1]:.2f} rad ≈ {phase_shifts[-1]/np.pi:.2f} π")

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(phase_shifts, marker='o')
plt.title("Phase Shift Across Cells (Peak-based)")
plt.xlabel("Cell Index")
plt.ylabel("Phase Shift (radians)")
plt.grid(True)

# Also show total phase shift
total_shift = phase_shifts[-1] - phase_shifts[0]
print(f"Total phase shift: {total_shift:.2f} rad ≈ {total_shift / np.pi:.2f} π")

In [5]:
import numpy as np 

d_diff = np.array([[0,0,0,1,1,1], [1,1,0,1,1,1], [1, 0, 0, 0, 1, 1]])
first_zero_index = np.zeros(3, dtype=int)
last_zero_index = np.zeros(3, dtype=int)

for i in range(3): 
    if np.any(d_diff[i] == 0):
        print(d_diff[i])
        zero_indices = np.where(d_diff[i] == 0)[0]
        first_zero_index[i] = zero_indices[0]
        last_zero_index[i] = zero_indices[-1]

print('first zero index:', first_zero_index)
print('last zero index:', last_zero_index)


[0 0 0 1 1 1]
[1 1 0 1 1 1]
[1 0 0 0 1 1]
first zero index: [0 2 1]
last zero index: [2 2 3]
